<a href="https://colab.research.google.com/github/RishabhMishra19/projects/blob/master/scrap/scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup as BS
import requests
import lxml
import pandas as pd

def get_flipkart_prod_links(search_string):
        links=[] 
        try:
            page=requests.get('https://www.flipkart.com/search?q='+search_string).text
            soup=BS(page,'lxml')
            part1=soup.body.div.div.contents[2]#getting product section
            part2=part1.contents[1].contents[0].contents[1].contents#getting all prod section
            l=part2[0].text.split(' ')
            for i in range(len(l)):
                if l[i]=='of':
                    items=40
                    break
            
            base_url="https://www.flipkart.com"
            if items==24: 
                for prod in part2[1:-2]:
                    prod_link=prod.find('a').get('href')
                    full_link=base_url+prod_link
                    links.append(full_link)
            elif items==40:
                for prod in part2[1:-2]:
                    for pr in prod.div.contents:
                        prod_link=pr.find('a').get('href')
                        full_link=base_url+prod_link
                        links.append(full_link)
            else:
                pass
            return links
        except:
            return links

def prod_details(prod_link):
        page=requests.get(prod_link).text
        soup=BS(page,'lxml')
        part1=soup.body.div.div.contents[2]
        part2=part1.contents[1].find('div').contents[1].contents
        
        detail_part=part2[1]
        comment_part=part2[-1]
        
        name=detail_part.div.contents[0].text
        for content in detail_part.div.contents:
            try:
                if content.contents[0].get('class')[0]=='_2i1QSc':
                    price=content.text.split('₹')[1]
                    break
            except:
                pass
            
        
        for content in detail_part.div.contents:
            try:
                if content.contents[0].get('class')[0]=='_3ors59':
                    overall_rating=content.span.text
                    num_ratings=content.text.split(' ')[0][len(overall_rating):]
                    num_reviews=content.text.split(" ")[-2].replace(",","")
                    if not num_reviews.isdigit():
                        num_reviews=num_reviews.split('&')[1]
                    break
            except:
                pass
            
        base_url="https://www.flipkart.com"
        details=[]
        try:
            for content in comment_part.contents[::-1]:
                if content.div.contents[-1].name=='a':
                    all_reviews_link=content.div.contents[-1].get('href')
                    all_reviews_full_link=base_url+all_reviews_link
                    details=[name,price,overall_rating,num_ratings,num_reviews,all_reviews_full_link]
                    break
        except:
            pass

        return details

def review_details(review_link):
        page_p=requests.get(review_link).text
        soup_p=BS(page_p,'lxml')
        part1_p=soup_p.body.div.div.contents[2]
        part2_p=part1_p.find('div').div.contents[1].contents
        pages=int(part2_p[-1].find('span').text.split(' ')[-1].replace(",",""))
        
        if pages>5:
            pages=5
        Reviews=[]
        
        for i in range(pages):
            link=review_link+f'&page={i+1}'
            page=requests.get(link).text
            soup=BS(page,'lxml')
            part1=soup.body.div.div.contents[2]
            part2=part1.div.div.contents[1].contents
            reviews_part=part2[2:-1]
            
            for Review in reviews_part:
                review_head=Review.div.div.div.contents[0]
                review_heading=review_head.p.text
                review_rating=review_head.div.text
                review=Review.div.div.div.contents[1].text[:-9]
                Reviews.append([review_rating,review_heading,review])
        return Reviews

def all_details(search_string):
        k=0
        df=pd.DataFrame(columns=['prod_name','prod_price','overall_rating','num_ratings','num_reviews','user_rating','user_review_heading','user_review_content'])
        
        search_item_links=get_flipkart_prod_links(search_string)

        if len(search_item_links)==0: 
            links=search_item_links
        else:
            links=search_item_links[:10]
        for item_link in links:
            
            item_details=prod_details(item_link)
            
            try:
                reviews_list=review_details(item_details[-1])
                for item_review in reviews_list:
                    k=k+1
                    df.loc[k]=[item_details[0],item_details[1],item_details[2],item_details[3],item_details[4],item_review[0],item_review[1],item_review[2]]
                    if k>=100:
                        return df
            except:
                pass
        return df



In [0]:
#search anything in here just enter string
all_details('bed')

,prod_name,prod_price,overall_rating,num_ratings,num_reviews,user_rating,user_review_heading,user_review_content
1,Flipkart Perfect Homes Opus Engineered Wood Qu...,"12,490",4,"8,558",1407,5,Fabulous!,Good Product from Flipkart. I have purchased K...
2,Flipkart Perfect Homes Opus Engineered Wood Qu...,"12,490",4,"8,558",1407,3,Does the job,The product is not very good . it is just ok. ...
3,Flipkart Perfect Homes Opus Engineered Wood Qu...,"12,490",4,"8,558",1407,4,Good choice,I got it in 10313 King size bed. Quality of th...
4,Flipkart Perfect Homes Opus Engineered Wood Qu...,"12,490",4,"8,558",1407,3,Nice,Reviewing after using 15 days what i observed ...
5,Flipkart Perfect Homes Opus Engineered Wood Qu...,"12,490",4,"8,558",1407,5,Awesome,Thankyou and God Bless You All for the Excelle...
...,...,...,...,...,...,...,...,...
96,Flipkart Perfect Homes Carol Engineered Wood Q...,"10,490",4,"6,559",973,4,Pretty good,Amazing bed for the price. Sturdy and comforta...
97,Flipkart Perfect Homes Carol Engineered Wood Q...,"10,490",4,"6,559",973,4,Moderate quality,"In this price the bed is pretty good, and the ..."
98,Flipkart Perfect Homes Carol Engineered Wood Q...,"10,490",4,"6,559",973,3,Value-for-money,"Look wise good, quality need to improved after..."
99,Flipkart Perfect Homes Carol Engineered Wood Q...,"10,490",4,"6,559",973,5,Best in the market!,As usual flipkart delivered on schedule. I hav...


In [0]:
all_details('sofa')

,prod_name,prod_price,overall_rating,num_ratings,num_reviews,user_rating,user_review_heading,user_review_content
1,Muebles Casa Croma Leatherette 3 Seater Sofa ...,"18,990",4.2,"1,201",231,5,Perfect product!,I was a bit sceptical about buying sofa from o...
2,Muebles Casa Croma Leatherette 3 Seater Sofa ...,"18,990",4.2,"1,201",231,5,Classy product,Nice sofa..... I like it...... Its cover som...
3,Muebles Casa Croma Leatherette 3 Seater Sofa ...,"18,990",4.2,"1,201",231,5,Simply awesome,The sofa was delivered right on time and insta...
4,Muebles Casa Croma Leatherette 3 Seater Sofa ...,"18,990",4.2,"1,201",231,4,Nice product,it's a very bad experience for me to buy a fur...
5,Muebles Casa Croma Leatherette 3 Seater Sofa ...,"18,990",4.2,"1,201",231,4,Good quality product,"when i sat on this sofa i feel very happy, the..."
...,...,...,...,...,...,...,...,...
96,Muebles Casa Marco Leatherette 6 Seater Sofa ...,"34,390",4.4,"1,052",212,5,Must buy!,"value for money, quick delivery and great desi..."
97,Muebles Casa Marco Leatherette 6 Seater Sofa ...,"34,390",4.4,"1,052",212,5,Super!,nice product in a reasonable price...
98,Muebles Casa Marco Leatherette 6 Seater Sofa ...,"34,390",4.4,"1,052",212,5,Best in the market!,A good value sofa.The only setback that it is ...
99,Muebles Casa Marco Leatherette 6 Seater Sofa ...,"34,390",4.4,"1,052",212,5,Just wow!,This l shape sofa is just awesome!! Its pillow...


# New Section